# funcion para obtener informacion sin limite de datos

In [1]:
import requests
import json
from requests.auth import HTTPBasicAuth
import time
import pandas as pd
import re

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

def elasticScroll(elasticParameters, query, pages):
    # parametros de salida
    # parametros del indice
    elasticURL = elasticParameters["elasticURL"]
    elasticIndex = elasticParameters["elasticIndex"]
    elasticUser = elasticParameters["elasticUser"]
    elasticPassword = elasticParameters["elasticPassword"]
    
    if(len(elasticURL)==0) or (len(elasticIndex)==0) or (len(elasticUser)==0) or (len(elasticPassword)==0):
        raise Exception("Revisa los parametros")
    # se define la url que apunta al indice de elastic
    url_search = f"{elasticURL}/{elasticIndex}/_search?scroll=1m"
    # se ejecuta la query
    response = requests.get(url_search, json=query, auth=HTTPBasicAuth(elasticUser, elasticPassword))
    # retorna una lista con el resultado de la query
    search = json.loads(response.text)
    # guardamos el scroll id correspondiente a la query
    scroll_id = search["_scroll_id"]
    # url scroll
    url_scroll = f"{elasticURL}/_search/scroll"
    scroll_query = {
                "scroll": "1m",
                "scroll_id": f"{scroll_id}"
            }

    # condiciones iniciales
    scroll_search = {"hits":{"hits":1}}
    if pages:
        # hay paginacion
        # condiciones iniciales
        from_ = pages["from"]
        size_ = pages["size"]
        count = len(search["hits"]["hits"])

        while scroll_search["hits"]["hits"] and count < from_ + size_:
            scroll_response = requests.get(url_scroll, json=scroll_query, auth=HTTPBasicAuth(elasticUser, elasticPassword))
            scroll_search = json.loads(scroll_response.text)
            if not scroll_search["hits"]["hits"]:
                continue
            else:
                search["hits"]["hits"].extend(scroll_search["hits"]["hits"])
            count += len(scroll_search["hits"]["hits"])

        search["hits"]["hits"] = search["hits"]["hits"][from_:from_+size_+1]

    else:
        # Se devuelven todos los resultados
        while scroll_search["hits"]["hits"]:
            scroll_response = requests.get(url_scroll, json=scroll_query,
                                           auth=HTTPBasicAuth(elasticUser, elasticPassword))
            scroll_search = json.loads(scroll_response.text)
            if not scroll_search["hits"]["hits"]:
                continue
            else:
                search["hits"]["hits"].extend(scroll_search["hits"]["hits"])
    # Se elina el campo scroll del scroll_body
    del scroll_query["scroll"]
    # Se elimina el scroll de elasticsearch para liberar memoria
    delete = requests.delete(url_scroll, json=scroll_query, auth=HTTPBasicAuth(elasticUser, elasticPassword))
    return search

In [2]:
# definiendo parametros de entrada

In [3]:
# datos de conexion a elastic
elasticParameters = {"elasticURL": "https://es-dev.e-contact.cl"
                    , "elasticIndex": "lea_sequences-events-banco_de_chile" 
                    , "elasticUser": "jcalderon"
                    , "elasticPassword": "jcalderon123"
                    }

# query custom para obtener fechas
query = {
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        {
            
          "range": {
            "interactionData.dateTimeUTC": {
              "format": "strict_date_optional_time",
              "gte": "2022-05-26T00:00:00.000Z",
              "lte": "2022-05-30T00:00:00.000Z"
            }
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}

# total de paginas
pages =  {
    "from": 0,
    "size": 5000
}

# consulta a indice

In [4]:
response = elasticScroll(elasticParameters, query, pages)

In [5]:
def limpiar_texto(texto):
    """
    Función para realizar la limpieza de un texto dado.
    """
    # Eliminamos los caracteres especiales
   # texto = re.sub(r'\W', ' ', texto)
    # Eliminado las palabras que tengo un solo caracter
    texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', str(texto))
    # Sustituir los espacios en blanco en uno solo
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    replacements = (
        ("á", "a"),("é", "e"),("í", "i"),("ó", "o"),("ú", "u"),
        ("ñ", "n"),("?", ""),("¿", ""),("%", ""),("$", ""),
        ("#", ""),("&", ""),("(", ""),(")", ""),("=", ""),
        ("¡", ""),("!", ""),("*", ""),("+", ""),("~", ""),
        ("[", ""),("]", ""),("}", ""),("{", ""),("^", ""),
        ("<", ""),(">", ""),("¬", ""),("¨", ""),("_", "")
    )
    
    for a, b in replacements:
        texto = texto.replace(a, b)
    # Convertimos textos a minusculas
    #texto = texto.lower()
    return texto

# transformar response (dict) a dataframe

In [6]:
from pandas import DataFrame, json_normalize
import numpy as np

df = json_normalize(json.loads(json.dumps(response))["hits"]["hits"])
df_00 = df [['_source.conversationSequence.topicName','_source.interactionData.interactionId','_source.interactionData.dateTimeUTC']]
df_00['topicName']= df_00['_source.conversationSequence.topicName']
df_00['interactionId']= df_00['_source.interactionData.interactionId']
df_00['Fec']= df_00['_source.interactionData.dateTimeUTC']
df_00['hora']= pd.to_datetime(df_00['Fec'].apply(str).str.slice(start = 0, stop = 13))
df_00['fecha'] = pd.to_datetime(df_00['Fec']).dt.date
df_00['hora'] = pd.to_datetime(df_00['hora']).dt.time
df_00['dia'] = pd.to_datetime(df_00['fecha'])
df_00['dia'] = df_00['dia'].dt.day_name()
df_00["topicName"] = df_00["topicName"].replace({ "": "NO_IDENTIFICADO", " ": "NO_IDENTIFICADO" })
df_00 = df_00[['topicName','interactionId','fecha','hora','dia']]
#df_00['key'] =df_00['topicName'] + ',' + df_00['interactionId'] + ',' + str(df_00['fecha']) + ',' + str(df_00['hora'])+ ',' + df_00['dia']
df_00 = df_00.drop_duplicates()

df_grafo2 = pd.DataFrame({'topicName': ['Corte llamado', 'Red sucursal','Reclamación','Escalamiento de llamada','Reincidencia'], 'matched': True})

# crea df 01 agrupando y contando por columna interactionId
df_01=df_00.groupby(["interactionId"]).size().reset_index(name='cantidad')

# concatena ambos df (00 y 01) para crear df 02
df_02 = df_00.merge(df_01, how='inner', on='interactionId')


#######################################################################

# filtro para aplicar en el df 02 cuando el campo cantidad == 1
filter01 = df_02["cantidad"]==1
df_02.where(filter01, inplace = True)

# crea df 03 apartando aquellos registros Nan
df_03 = df_02[df_02['cantidad'].notna()]

##########################################################################

# crea df 04 agrupando y contando por columna interactionId
df_04=df_00.groupby(["interactionId"]).size().reset_index(name='cantidad')

# concatena ambos df (00 y 04) para crear df 05
df_05 = df_00.merge(df_04, how='inner', on='interactionId')

# filtro para aplicar en el df 05 cuando el campo cantidad != 1
filter02 = df_05["cantidad"]!=1
df_05.where(filter02, inplace = True)

# crea df 06 apartando aquellos registros Nan
df_06 = df_05[df_05['cantidad'].notna()]

# crea df  apartando aquellos registros cuyo topicName == 'NO_IDENTIFICADO'
new_df = df_06[~df_06['topicName'].str.contains("NO_IDENTIFICADO")]

# concatena ambos df (03 y new_df) para crear df 01
df_00 = pd.concat([df_03, new_df], axis=0)

# elimina col cantidad
del df_00['cantidad']

df_00['contador']=1

df_00['topicName'] = df_00.topicName.apply(limpiar_texto)

############################ AGRUPA SEGUN REQUERIMIENTO (FECHA),(FECHA,HORA),(FECHA,HORA,DIA) ############################
#agrupadoFecha = df_00.groupby(['topicName','fecha']).agg({'contador': 'count'}).reset_index()
#agrupadoFecha['cantidad'] = agrupadoFecha['contador']
#del agrupadoFecha['contador']
#agrupadoFecha['total'] = agrupadoFecha['cantidad'].sum()
#agrupadoFecha = df_grafo2.merge(agrupadoFecha, how='right', on='topicName')
#agrupadoFecha['grafo']=agrupadoFecha['matched']
#del agrupadoFecha['matched']
#agrupadoFecha['grafo']=agrupadoFecha['grafo'].apply(lambda x: 1 if x!=1 else 0)
#agrupadoFecha['grafo']=agrupadoFecha['grafo'].replace({0:2})

#agrupadoFecha.to_csv("C:\\Users\\jcalderon\\Desktop\\Trafico\\Python_2021\\ProyectoBCH\\" + 'Total'+'.csv', index = False)

#agrupadoFechaHora = df_00.groupby(['topicName','fecha','hora']).agg({'contador': 'count'}).reset_index()
#agrupadoFechaHora['cantidad'] = agrupadoFechaHora['contador']
#del agrupadoFechaHora['contador']
#agrupadoFechaHora_A = agrupadoFechaHora.groupby(['fecha','hora']).agg({'cantidad': 'sum'}).reset_index()
#agrupadoFechaHora_A['total'] = agrupadoFechaHora_A['cantidad']
#del agrupadoFechaHora_A['cantidad']
#agrupadoFechaHora = agrupadoFechaHora.merge(agrupadoFechaHora_A, how='inner', on=['fecha','hora'])
#agrupadoFechaHora['final'] =((agrupadoFechaHora['cantidad']) / agrupadoFechaHora['total'])
#agrupadoFechaHora = df_grafo2.merge(agrupadoFechaHora, how='right', on='topicName')
#agrupadoFechaHora['grafo']=agrupadoFechaHora['matched']
#del agrupadoFechaHora['matched']
#agrupadoFechaHora['grafo']=agrupadoFechaHora['grafo'].apply(lambda x: 1 if x!=1 else 0)
#agrupadoFechaHora['grafo']=agrupadoFechaHora['grafo'].replace({0:2})

#agrupadoFechaHora.to_csv("C:\\Users\\jcalderon\\Desktop\\Trafico\\Python_2021\\ProyectoBCH\\" + 'Total'+'.csv', index = False)

agrupadoFechaHoraDia = df_00.groupby(['topicName','fecha','hora','dia']).agg({'contador': 'count'}).reset_index()
agrupadoFechaHoraDia['cantidad'] = agrupadoFechaHoraDia['contador']
del agrupadoFechaHoraDia['contador']
agrupadoFechaHoraDia_A = agrupadoFechaHoraDia.groupby(['fecha','hora','dia']).agg({'cantidad': 'sum'}).reset_index()
agrupadoFechaHoraDia_A['total'] = agrupadoFechaHoraDia_A['cantidad']
del agrupadoFechaHoraDia_A['cantidad']
agrupadoFechaHoraDia = agrupadoFechaHoraDia.merge(agrupadoFechaHoraDia_A, how='inner', on=['fecha','hora','dia'])
agrupadoFechaHoraDia['final'] =((agrupadoFechaHoraDia['cantidad']) / agrupadoFechaHoraDia['total'])
agrupadoFechaHoraDia = df_grafo2.merge(agrupadoFechaHoraDia, how='right', on='topicName')
agrupadoFechaHoraDia['grafo']=agrupadoFechaHoraDia['matched']
del agrupadoFechaHoraDia['matched']
agrupadoFechaHoraDia['grafo']=agrupadoFechaHoraDia['grafo'].apply(lambda x: 1 if x!=1 else 0)
agrupadoFechaHoraDia['grafo']=agrupadoFechaHoraDia['grafo'].replace({0:2})


agrupadoFechaHoraDia.to_csv("C:\\Users\\jcalderon\\Desktop\\Trafico\\Python_2021\\ProyectoBCH\\" + 'Total'+'.csv', index = False)